# Exercise 5 - Automatically find Exchange links for DWM Query

In this demo we will see how to combine DWM and Iknaio to automatically find connections to exchanges given a set of crypto addresses mentioned in some genre of darkweb sites. Our topic today is CSAM.

## Preparations

First, we install the graphsense-python package and define an API-key. An API-key for the [GraphSense](https://graphsense.github.io/) instance hosted by [Iknaio](https://www.ikna.io/) can be requested by sending an email to [contact@iknaio.com](contact@iknaio.com).

In [1]:
!pip install graphsense-python seaborn tqdm json-api-doc openpyxl

import shelve
import os
import json
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import graphsense
from graphsense.api import bulk_api, general_api


# expect some local libs 
# files can be found at (https://github.com/iknaio/iknaio-api-tutorial/tree/main/standalone)
# in google colab they are downloaded automatically
try:
    from google.colab import userdata
    libs = ["dwm", "gs"]
    for l in libs:
        response = requests.get(f"https://raw.githubusercontent.com/iknaio/iknaio-api-tutorial/refs/heads/main/standalone/{l}.py")
        with open(f"{l}.py", "w") as f:
            f.write(response.text)
except ImportError:
    pass

import dwm
import gs

# We only work with BTC in this example
CURRENCY = 'btc'
BASE_PATH = ""


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Load credentials for Iknaio and Dwm

In [2]:
# find Secrets to use
try:
    from google.colab import userdata
    
    # Expected structure of the file
    secrets = {
         "gs-api-key" : userdata.get('gs-api-key'),
         "dwm-credentials" : {"username": userdata.get('dwm-credentials-username'), "password": userdata.get('dwm-credentials-password')}
    }
    print("Found Colab Secrets")
except ImportError:
    with open("secrets.json") as f:
        secrets = json.load(f)
        print("Found secrets.json")

Found secrets.json


## Connect to Google Drive
Uncomment code if you want to persist you results in google drive.

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

# 1. DWM - Load addresses mentioned on Darkweb sites

In [4]:
# Request authentication token
headers = dwm.authenticate_api(secrets['dwm-credentials'])

## 1.a Load domains with a specific title

In [5]:
# Collect domains related to title
title = "Alice with violence CP"

df_domains_all = dwm.get_domains_by_title(title, headers)
df_domains_all[["domain_url", "title", "status", "discovered_at"]].head(10)

Processed 1 out of 3 pages
Processed 2 out of 3 pages
Processed 3 out of 3 pages


,domain_url,title,status,discovered_at
0,http://x5w2vdx4lmvha27xjgnnnceudiqd6f3gjuegadu...,Alice with violence CP,online,2024-10-29T20:28:53.000Z
1,http://x5cj2bvcxngjohqi7hpkf67fqbqg7wkptcqa2sa...,Alice with violence CP,online,2024-10-29T20:28:53.000Z
2,http://vvniruuxyborklcc3i7s5mlerjuysw2rwrd6svr...,Alice with violence CP,online,2024-10-29T20:28:53.000Z
3,http://bv34z4lb4mr7djs7y7y62db6pocnmwoa7suxs4o...,Alice with violence CP,online,2024-10-29T20:28:53.000Z
4,http://bzuk5hv4r2z3n3asimysuxzwctm75eq3fzcd2ah...,Alice with violence CP,online,2024-10-29T20:28:53.000Z
5,http://c32rldfwe67vlgmhdlchpimndyrp4n35hxhlynu...,Alice with violence CP,online,2024-10-29T20:28:53.000Z
6,http://cig5yp2z4byvffhy4sx6b6e3za5mimqgdbqyj7y...,Alice with violence CP,online,2024-10-29T20:28:53.000Z
7,http://cl4a25itvp53xtuxxpftm4qp2swyoj27vllfwx2...,Alice with violence CP,online,2024-10-29T20:28:53.000Z
8,http://dptvuwjqhoswduhse4hn3oygft7xf23mkqmrblw...,Alice with violence CP,online,2024-10-29T20:28:53.000Z
9,http://aojsktg6obv2axoq57b44iugh4n5wkbvxxc5a33...,Alice with violence CP,online,2024-10-29T20:28:53.000Z


### Stats on domains

In [6]:
# only keep offline domains
# initial exploration has shown that flows from online websites all go trough service (tbdice)
# thus are hard to trace
df_domains = df_domains_all # .query("status!='online'")
nr_domains = len(df_domains)
print(f"We have found {nr_domains} domains with title: {title}")

We have found 2383 domains with title: Alice with violence CP


## 1.b Get crypto addresses on the domains

In [7]:
# Caution long-running (takes up to 1h)
df_cryptos_all_file = BASE_PATH + "dwm_addresses_per_domain.pkl"
if os.path.exists(df_cryptos_all_file):
    df_cryptos_all = pd.read_pickle(df_cryptos_all_file)
else:
    df_cryptos_all = dwm.get_crypto_addresses_for_domains(df_domains, headers)

    # save data since its long running
    df_cryptos_all.to_pickle(df_cryptos_all_file)

df_cryptos_all.head(5)

,domain_id,crypto_asset_id,type,address,appearances,discovered_at
0,15979413,11623431,BTC,374zYn5D8HXvfwnRxVY7pw2onPGH7fJTzn,52,2024-10-29T17:28:29.000Z
1,15979418,11652194,BTC,3NHLkPVKMHVa3dXaGRcu85E4kyCU4ocuho,44,2024-12-03T07:33:14.000Z
2,15979418,11623465,BTC,3N7NQ5byrFPsUxHoTqUhgwRLtEUiHLhx5D,44,2024-10-29T18:53:08.000Z
3,15979419,11642445,BTC,377VBtpXz3Ed6WZcwzgD4CCUX5nbMpNQTT,69,2024-11-15T02:27:36.000Z
4,15979419,11623474,BTC,358PjkVHb85B4VnvsrMt9LAKxcbnHtCaYk,62,2024-10-29T19:45:03.000Z


### Only keep BTC addresses

In [8]:
df_cryptos = df_cryptos_all.query(f"type=='{CURRENCY.upper()}'")
unique_addresses = len(df_cryptos["address"].unique())
print(f"We have found {len(df_cryptos)} BTC-addresses on these domains {unique_addresses} of which are unique")

We have found 37093 BTC-addresses on these domains 23629 of which are unique


### Save results

In [9]:
# save output in an excel file
with pd.ExcelWriter(BASE_PATH + "alice_dwm.xlsx") as writer:
    df_domains.to_excel(writer, sheet_name="Domains", index=False)
    df_cryptos.to_excel(writer, sheet_name="Crypto-Assets", index=False)

# Save unique addresses in a CSV file
df_cryptos[["address"]].drop_duplicates(subset=["address"]).to_csv(BASE_PATH + "addresses.csv")

# 2. Iknaio - Inspecting addresses and finding Exit-points (exchanges)

Our main goal in this section is to find direct or if none exist, indirect paths to exchanges. Centralized exchanged are required to implement KYC procedures. Thus they usually know a passport number or similar identifying information about their customers.

In [10]:
# setup connection to graphsense
configuration = graphsense.Configuration(
    host = "https://api.ikna.io/",
    api_key = {
        'api_key': secrets["gs-api-key"]
    }
)

# Test Connection
with graphsense.ApiClient(configuration) as api_client:
    api_instance = general_api.GeneralApi(api_client)
    api_response = api_instance.get_statistics()
    display(pd.DataFrame([ {"name": x['name'], "timestamp": gs.ts_to_pds(x['timestamp'])} for x in api_response['currencies']]))

,name,timestamp
0,btc,2025-01-13 17:37:24
1,bch,2025-01-13 15:34:30
2,ltc,2025-01-13 18:03:44
3,zec,2025-01-13 15:21:53
4,eth,2025-01-13 17:49:59
5,trx,2025-01-11 06:42:12


# 2.a How many of the found addresses are used?

Instead of querying each address individually, we just pass the dataframe of the known addresses.

In [11]:
seed_addresses = pd.read_csv(BASE_PATH + "addresses.csv")

respAddrDF = gs.get_csv(configuration, "get_address", CURRENCY, {
                    'address': seed_addresses['address'].to_list()
                })

used_addresses = respAddrDF[["address", "balance_eur", "total_received_eur", "total_spent_eur", "in_degree", "out_degree", "no_incoming_txs", "no_outgoing_txs", "first_tx_timestamp", "last_tx_timestamp", "entity"]].dropna()
used_addresses[["address"]].to_csv(BASE_PATH + "used_addresses.csv")
used_addresses.head(5)

,address,balance_eur,total_received_eur,total_spent_eur,in_degree,out_degree,no_incoming_txs,no_outgoing_txs,first_tx_timestamp,last_tx_timestamp,entity
23284,3QVpg7Syfad3Epz7HgRSggvcWctR11vxY4,0.0,34.05,18.84,4.0,1.0,1.0,1.0,1.633726e+09,1.681962e+09,874019470.0
23285,3LPcdpX9TR6TXYsyXHnm15dYUahtwzDcy2,0.0,30.90,21.68,1.0,1.0,1.0,1.0,1.641542e+09,1.681962e+09,920681978.0
23286,3PuUSd9vAR8cWgdsWdgR7pmqAU3D3K4iRW,0.0,33.63,21.42,1.0,1.0,1.0,1.0,1.613450e+09,1.681962e+09,760381701.0
23287,3N1FR7zGDf56kAnV1Nso4aXW2eWYVUVpTH,0.0,32.17,28.39,1.0,1.0,1.0,1.0,1.610328e+09,1.681962e+09,760381701.0
23288,3EXT4nELaXxvJUWFXumqyhowpT77quJKKM,0.0,30.74,21.03,1.0,1.0,1.0,1.0,1.642263e+09,1.681962e+09,920681978.0


In [12]:
print(f"{len(used_addresses)} addresses received {sum(used_addresses['total_received_eur']):.2f} EUR, Balance {sum(used_addresses['balance_eur']):.2f} EUR")
print(f"Activity period of the addresses was: {gs.ts_to_pds(min(used_addresses['first_tx_timestamp']))} to {gs.ts_to_pds(max(used_addresses['last_tx_timestamp']))}")

345 addresses received 32192.64 EUR, Balance 642.38 EUR
Activity period of the addresses was: 2019-12-13 13:14:24 to 2025-01-04 22:50:09


# 2.b Are there direct links to exchanges?

Direct link here means without any hops in-between directly form the source address found in the darkweb.

In [13]:
respAddrNbrDF = gs.get_csv(configuration, "list_address_neighbors", CURRENCY, {
                    'address': used_addresses['address'].to_list(),
                    'direction': 'out',
                    'include_labels': True
                })

with_label_direct = respAddrNbrDF.query("labels.notnull()")

with_outgoing_direct = respAddrNbrDF.query("_info != 'no data'")

print(f"We have found {len(with_outgoing_direct)} outgoing neighbors, {len(with_label_direct)} are known")

with_label_direct[["_request_address", "address_address", "labels"]].head(50)

We have found 621 outgoing neighbors, 46 are known


,_request_address,address_address,labels
19,3FWgtSB2rfMjtP7dVUxSfb2WTTk1APVwye,3FtduGyvPXsJV3Z44MgRThoensbNopWE6Z,Dark Web
24,bc1qwt7n66u465rfwf4gqwk20ywxlqwmrv8zkwf03s,bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq,Binance
58,3681ymf82FpntQ75xubrVBsFGmkNyujc3z,3FtduGyvPXsJV3Z44MgRThoensbNopWE6Z,Dark Web
60,14smqvFY4PgMpjmAHmfg4X824bh5to6HXn,1QJcNN3TdF5yQmbbXJtuBTtoeNABRJ2hbJ,Dark Web
93,3PJrBxShw96D5s5xDy9FeEaF3ukiBMj5iz,bc1qlduqs0yax4zftfuez8frvpenlxk9gmfpmaayyr,Wasabi
94,1AiQPyYhM9yQhiYwDmPA9K98pqjb8wyNHy,16cYCNrJXvwA3LVTCPFkew9VCKvwbqYh7Z,Dark Web
95,1AiQPyYhM9yQhiYwDmPA9K98pqjb8wyNHy,1J2UUxkuB7YUgHKGHuJYCHqQQUozWnne6P,Dark Web
120,3Cm4s3naBDELk8JUxh5s43fQ73wr1mZwB7,3FtduGyvPXsJV3Z44MgRThoensbNopWE6Z,Dark Web
149,1PmqwTh14wwWET7mqmwujWK5ook93e2bD6,1FGob8526nQDxpHWgmB69JfLZnnCrZQha1,bitcoinabuse_ransomware
158,3L7S3aDjdHexMoBmAx3Pap2JfocTE5efBc,bc1qlduqs0yax4zftfuez8frvpenlxk9gmfpmaayyr,Wasabi


In [14]:
binance_direct = gs.get_csv(configuration, "list_address_links", CURRENCY, {
                    'address': ["bc1qwt7n66u465rfwf4gqwk20ywxlqwmrv8zkwf03s"],
                    'neighbor': ['bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq'],
                })
binance_direct

for i, r in binance_direct.iterrows():
    print(f"https://app.ikna.io/pathfinder/{r['currency']}/path/PA_{r['_request_address']},T_{r['tx_hash']},HA_{r['_request_neighbor']}")

https://app.ikna.io/pathfinder/btc/path/PA_bc1qwt7n66u465rfwf4gqwk20ywxlqwmrv8zkwf03s,T_d9344a24b30d9650bfaeb116f32203f732faa0205d1d42d0d0ab4dbd6be58611,HA_bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq


# 2.c Can I find links to exchange via Clusters?

We now fetch the cluster for each address

In [15]:
respEntityDF = gs.get_csv(configuration, "get_entity", CURRENCY, {
                                     'entity': used_addresses['entity'].drop_duplicates().to_list(),
                                     "exclude_best_address_tag": True
                                     })

clusters = respEntityDF[
    ["best_address_tag_label",
     "root_address",
     "no_addresses",
     "best_address_tag_label",
     "balance_eur",
     "total_received_eur",
     "total_spent_eur",
     "first_tx_timestamp",
     "last_tx_timestamp"]
     ]

print(f"~ {sum(clusters['no_addresses'])-len(used_addresses)} new addresses have been found. In {len(clusters)} clusters. They received {sum(clusters['total_received_eur']):.2f} EUR, Balance {sum(clusters['balance_eur']):.2f} EUR")
print(f"Activity period of the cluster addresses were: {gs.ts_to_pds(min(clusters['first_tx_timestamp']))} to {gs.ts_to_pds(max(clusters['last_tx_timestamp']))}")
# clusters.query("best_address_tag_label.notnull()")
clusters.head(5)

~ 12726 new addresses have been found. In 91 clusters. They received 1378571.12 EUR, Balance 6607.20 EUR
Activity period of the cluster addresses were: 2014-08-09 23:48:57 to 2025-01-12 13:50:05


,best_address_tag_label,root_address,no_addresses,best_address_tag_label,balance_eur,total_received_eur,total_spent_eur,first_tx_timestamp,last_tx_timestamp
0,NaN,1G4AWvMwrckZFCoVGc86DQQQmxRrFLhTE8,261,NaN,39.36,44688.99,45973.85,1567405055,1633543562
1,NaN,384sJZNtNAfrX48Hk8cKvGAWorCv2UpGXQ,46,NaN,0.00,949.91,948.35,1686765960,1690897018
2,NaN,bc1qusvxlt0kl3dh5vsdn2py395jnqjcmhxgam40tp,2,NaN,0.00,229.59,215.02,1645057562,1659472255
3,NaN,3BHqn4f41ee6Rhb7mXTsoL65BmyBsqjcRL,1,NaN,34.07,35.18,0.00,1735206829,1735206829
4,NaN,37JuBmYrKZQvS67LN8rotT6byZqUfGWDFt,2,NaN,0.00,153.22,204.85,1607213788,1639740665


In [16]:
print(f"The clusters combined has ~{respEntityDF['no_addresses'].sum()} addresses")

The clusters combined has ~13071 addresses


### Fetch indirect addresses (Clusters)

In [17]:
df_cluster_addresses_file = BASE_PATH + "ikn_cluster_addresses.pkl"
if os.path.exists(df_cluster_addresses_file):
    respEntityAddressesDF = pd.read_pickle(df_cluster_addresses_file)
else:
    respEntityAddressesDF = gs.get_csv(configuration, "list_entity_addresses", CURRENCY, {
                                        'entity': used_addresses['entity'].drop_duplicates().to_list(),
                                        })    
    # save data since its long running
    respEntityAddressesDF.to_pickle(df_cluster_addresses_file)

print(f"Got {len(respEntityAddressesDF)} addresses.")

respEntityAddressesDF.head(5)

Got 11321 addresses.


,_error,_info,_request_entity,actors,address,balance_eur,balance_usd,balance_value,currency,entity,...,status,token_balances,total_received_eur,total_received_usd,total_received_value,total_spent_eur,total_spent_usd,total_spent_value,total_tokens_received,total_tokens_spent
0,NaN,NaN,746611836.0,NaN,3617kaYQedXG8WhkPr7XhH79tDaoj2SYjd,0.0,0.0,0,btc,746611836,...,clean,NaN,28.00,33.96,176938,52.51,56.25,176938,NaN,NaN
1,NaN,NaN,746611836.0,NaN,3DDxMgfPW1WxrHTCdNqir3aZEHPFRSzTsz,0.0,0.0,0,btc,746611836,...,clean,NaN,28.48,34.54,180000,53.42,57.23,180000,NaN,NaN
2,NaN,NaN,746611836.0,NaN,3BCBAzZp7oG6aHztiBD7LnGzWYiH4ecgcY,0.0,0.0,0,btc,746611836,...,clean,NaN,33.01,40.42,177255,52.60,56.35,177255,NaN,NaN
3,NaN,NaN,746611836.0,NaN,35gJMmyvuaJR1QYRvvxVDhkchaRmiezuhp,0.0,0.0,0,btc,746611836,...,clean,NaN,28.31,34.71,150000,44.51,47.69,150000,NaN,NaN
4,NaN,NaN,746611836.0,NaN,3AbpkyDLimkVuJpoMQjZCAtNkPEcyWTRP5,0.0,0.0,0,btc,746611836,...,clean,NaN,33.48,41.04,150000,44.51,47.69,150000,NaN,NaN


### Fetch Nbrs of indirect addresses

In [18]:
df_cluster_addresses_nbrs_file = BASE_PATH + "ikn_cluster_addresses_nbrs.pkl"

if os.path.exists(df_cluster_addresses_nbrs_file):
    respEntityAddressesNbrsDF = pd.read_pickle(df_cluster_addresses_nbrs_file)
else:
    respEntityAddressesNbrsDF = gs.get_csv(configuration, "list_address_neighbors", CURRENCY, {
                        'address': respEntityAddressesDF['address'].to_list(),
                        'direction': 'out',
                        'include_labels': True
                    })
    
    respEntityAddressesNbrsDF.to_pickle(df_cluster_addresses_nbrs_file)

with_label_indirect = respEntityAddressesNbrsDF.query("labels.notnull()")

with_outgoing_indirect = respEntityAddressesNbrsDF.query("_info != 'no data'")

print(f"We have found {len(with_outgoing_indirect)} outgoing indirect neighbors, {len(with_label_indirect)} are known")

with_label_indirect[["_request_address", "address_address", "labels"]].head(50)

We have found 23136 outgoing indirect neighbors, 2583 are known


,_request_address,address_address,labels
16,1HbdYdx5iYp1QtvSHGNbbuSu7aXf4wDRTe,18vgt4H8TUBbmVcESLN8eP1AYt7Tf9GDhe,Dark Web
17,1DRmgsXpoBQMfH4ur1FMhMUCXGAA1Ef5kk,18vgt4H8TUBbmVcESLN8eP1AYt7Tf9GDhe,Dark Web
24,1FUmL13oEmHyGPx1rXLGdixBWVNmsG6wTL,19HHuqCe3Dihnyxo3ByUD6KEqExntwH2a5,Dark Web
25,1GFY7pkvKQGmCEvxLGDt6wpZNSy3gDbuiq,1Gh9j1mqUhURxySwBqAZswRw6U3TGi1da5,Dark Web
33,33HdXaShgsnHtAVDRpzuxCCy8zdH3UZ6gA,bc1qctuyq2ydygplrtx37rr97j2yqysm564darud8m,Wasabi
60,1Gnmo4oxhVsqzyYbqWXQSoPuFALBT8vepd,1JjKNuGQWf3ox3d5aH7QGx82mhyBvYGjKk,Dark Web
63,16P1svJuYY5jTHiP2pUV9h2DieXu2AMx2L,1CxJdbx6J6uGV3wntGQUB8MmBjZkmvddu7,Dark Web
77,1KG4vYKqmC295ZfpJpY1Dq9tpBEpa4RNMT,1D3YVSRcW1oRrXbdtgKS6c5xYkqrX24S9k,Dark Web
81,1HR4vC9M34wStuAwPMdA7Wo3uLANDvCZVh,139dKeRoidEZDf59mkW68jFyVDn3JgAjfX,Dark Web
88,12Q89z2qCUJindVrjBWk3gyuQ2i5WCGyUD,18ChnjX9npKVsivnZ4UWhE5gd5gwrYvKfE,Dark Web


There are a lot of entries, just show me the unique ones to get an overview.

In [19]:
list(with_label_indirect[["_request_address", "address_address", "labels"]]["labels"].unique())

['Dark Web', 'Wasabi', 'bitcoinabuse_ransomware', 'Binance']

Binance looks interesting lets filter for those.

In [20]:
with_label_indirect[["_request_address", "address_address", "labels"]].query("labels.str.contains('Binance')").head(100)

,_request_address,address_address,labels
1558,32hvFKXJLBLWwpNbEqyhMzfLv6YqynV2dV,bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq,Binance
2149,3GdEJT9sgHJRB43uY1vHh6f78kY7GcnFFd,bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq,Binance
2709,bc1qwt7n66u465rfwf4gqwk20ywxlqwmrv8zkwf03s,bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq,Binance
3603,37gApAfYKKWhVcaLE4DG9DH5nS76obPQHJ,bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq,Binance
3604,3QRqnHvFyULNrjbzxaR5T5jm1nt7t5HATA,bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq,Binance
...,...,...,...
16699,3Luvo8efh56LfieBdmepxZXdNoYrx65pmb,bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq,Binance
16926,3LiRrxJ191pycDj9Cu36SW6o8h9MPbeq1Z,bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq,Binance
19631,39RrUcRn8CrjtQDRZaQXtsp1hj1uAvwX1R,bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq,Binance
19879,3NsjyXMZNx9XG1eyyvdkMRNBecyVNH8WmZ,bc1qdyvxjc502ndn4ku7544hplqfmf0h70vyw5vpxq,Binance


# 2.d What if I look at multiple hops (using QL)? Are there any exchanges?

In [21]:
addresses_used_list = used_addresses["address"].to_list()

with shelve.open(BASE_PATH + 'trace_cache.db') as cache:
    x = gs.get_QL_results_many(addresses_used_list, CURRENCY, {"Authorization": secrets["gs-api-key"]}, cache)

relevant_traces = sorted([y for y in x if y['nr_pathes_found'] > 0], key=lambda x: x["pct_traced_to_exchange"])

print(f"Found {len(relevant_traces)} paths to exchange")

for p in relevant_traces:
    print(gs.get_Pathfinder_link_from_ql_result(p))

Searching centralized exchange connections for addresses: 100%|██████████| 345/345 [00:00<00:00, 140926.65it/s]

Found 2 paths to exchange
https://app.ikna.io/pathfinder/btc/path/PA_bc1q0780hyfqp2cf9cec5l8kn72sm502jkxz9ws0m3,T_ae180cd2e68977951bf25259ad81f1c0ed74d9ebcc46abaa46c5d620824bd7d3,HA_bc1q9qgt0340tylyhnpr8gxxhv3kg6m6mv3xtanr6naqtya447ejcn9sj36lwf,T_87c8319a1560c6288bf59892825063071a34d135267d01649c2705bcf70a41cd,HA_bc1quhruqrghgcca950rvhtrg7cpd7u8k6svpzgzmrjy8xyukacl5lkq0r8l2d
https://app.ikna.io/pathfinder/btc/path/PA_bc1q7zu3ffsnm55ltss0r0yj5vj6z5766ngdm34ast,T_1c81da30153892bec6cbe680634d83221030800d21d4e4a539eae31667363124,HA_bc1q8gqc2fmycy3mxw3pjjkwtawlspqwgvhgxrl5s0,T_0b68df2fc7bedc5cb1fd7cd27783b5b354ee8ac5e2bbcca087686786eb6874cf,HA_bc1q2scftr55wqxjggj34hu9z9hhz3emdtejy5a04q,T_9238d83ea4ff11f7f2adc86a75c0abd4ad8eb7cc0ebcc67459ea53a9e0b72ac3,HA_bc1qns9f7yfx3ry9lj6yz7c9er0vwa0ye2eklpzqfw
